In [43]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

movies = [
    {
        "movie": "Top Gun",
        "answer": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "answer": "👨‍👨‍👦🔫🍝",
    },
]

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

ex_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{movie}"),
        ("ai", "{answer}"),
    ]
)

ex_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=ex_prompt,
    examples=movies,
)

movie_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "When the user provides a movie title, respond with exactly three emojis that best represent the movie’s plot, themes, emotions, and key symbols. Not just the literal title. If the question is not a movie title, just answer what user said."),
        MessagesPlaceholder(variable_name="history"),
        ex_prompt,
        ("human", "{question}"),
    ]
)

def load_memory(input):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | movie_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({
        "question":movie,
    })
    memory.save_context({"input":movie}, {"output":result.content})



In [44]:
invoke_chain("Fight Club")

👊💼🤯

In [45]:
invoke_chain("Gravity")

🌌🚀😱

In [46]:
invoke_chain("In the Mood for Love")

🌸💔🎶

In [47]:
invoke_chain("What is the movie title I asked first time?")

Fight Club